# Assignment 1: Tri-gram Language Model and NER Tagging
Welcome to your first assignment of CSE-476! Your goal in this assignment is to implement a trigram language model, and then use its output as features to train a NER model using provided implementations of a perceptron model.

In [43]:
'''
Initial loading of the data file and the NLTK tokenizer.
Please do not modify this section.
You need to run this first.
'''
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from collections import defaultdict
nltk.download('punkt_tab')
nltk.download('brown')

# load brown corpus
def load_corpus():
    corpus = list(brown.sents())
    for i in range(len(corpus)):
        corpus[i] = " ".join(corpus[i])
    return corpus

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


## Task 1: Implement the TrigramLM class.

You are provided with some starting code. You are free to modify the starting code, as long as you meet all requirements as specified by the comments below, and your class can be used in the following way:


```
lm = TrigramLM("vocab.txt")
lm.train()
ranking = lm.next_word_ranking("this is a")
# Expected format of 'ranking':
# [("good", 0.04), ("matter", 0.03)....]
```

A few reviews of knowledge points:

- 1: Unknown tokens are tokens that are not in the vocabulary.

In [44]:
class TrigramLM:

    def __init__(self, vocab_file):
        self.vocabulary = []
        self.bigram_count_table = defaultdict(int)
        '''
        TODO
        Other than the given bigram_count_table, what else would you need?
        '''
        self.trigram_count_table = defaultdict(int)
        self.tokens = list()
        self.start_vocab = "<start>"
        self.end_vocab = "<end>"
        self.unknown_vocab = "<UNK>"
        self.load_vocab(vocab_file)

    def load_vocab(self, vocab_file):
        with open(vocab_file, 'r') as f:
            for line in f:
                self.vocabulary.append(line.strip())
        self.vocabulary.append(self.unknown_vocab)
        self.vocabulary.append(self.start_vocab)
        self.vocabulary.append(self.end_vocab)
        print(f"vocab loaded, size = {len(self.vocabulary)}")

    '''
    TODO
    Implement the tokenize function.
    @text is a string, e.g., text="Today is a good day"
    Return a list of strings of tokens, such as ["today", "is"...]
    1. You MUST use NLTK's word_tokenize() function to split text into tokens.
    2. You MUST implement a uncased LM. That is, the vocabularies in the given file are all lower-cased. You should lower-case all tokens here too.
    3. Think about what do you need to do with unknown_vocab?
    '''
    def tokenize(self, text):
        tokens = word_tokenize(text)
        tokens = [token.lower() if token.lower() in self.vocabulary else self.unknown_vocab for token in tokens]
        # Your code here.
        return tokens

    '''
    TODO
    Finish implementing the training function.
    This function takes the corpus, and iteratively build all counts that the model may need to rank next words.
    Think about what do you need to do with the start_vocab and end_vocab when loading data?
    '''
    def train(self):
        corpus = load_corpus()
        print(f"corpus loaded, size = {len(corpus)}")

        # Your code here.
        for i in corpus:
            tokens = self.tokenize(i)
            tokens = [self.start_vocab, self.start_vocab] + tokens + [self.end_vocab]

            for j in range(len(tokens)-2):
                t1, t2, t3 = tokens[j], tokens[j+1], tokens[j+2]
                self.bigram_count_table[f"{t1}, {t2}"] += 1
                self.trigram_count_table[f"{t1}, {t2}, {t3}"] += 1

    '''
    Implement the function that produces a list of top_n next words, given 'prior_context', with their probabilties.
    @prior_context: a string of the current context, e.g., "This is a", and the function tries to predict the next word.
    @top_n: returns the top-N most likely words.
    Return a list of top_n words with their probabilties, in the format of [("good", 0.04), ("matter", 0.03)....]
    Think about what do you need to do for start_vocab and end_vocab?
    '''
    def next_word_ranking(self, prior_context, top_n=10):
        # Your code here.
        words = self.tokenize(prior_context)
        bigram = None
        
        if len(words) >= 2:
            bigram = words[-2:]
        else:
            bigram = [self.start_vocab]*(2-len(words)) + words

        words_ranking = list()
        context_bigram = self.bigram_count_table.get(f"{bigram[0]}, {bigram[1]}", 0)

        if context_bigram > 0:
            for word in self.vocabulary:
                trigram = bigram + [word]
                context_trigram = self.trigram_count_table.get(f"{bigram[0]}, {bigram[1]}, {word}", 0)
                prob = context_trigram/context_bigram
                words_ranking.append([word, prob])
            words_ranking.sort(key=lambda x: (x[1], x[0]))
            words_ranking.reverse()
        else:
            words_ranking = words_ranking

        return words_ranking[:top_n]
        

## Task 2: Using the TrigramLM Predictions as Feature for NER.
You are given a data loading helper function that loads the training data for NER.

In [45]:
# !pip install datasets
from datasets import load_dataset

'''
Loading NER training data.
Please do not modify this section.
'''

def load_conll2003(split):
    # ignore other tags
    NER_tags={
        0: "O",
        1: "B-PER",
        2: "I-PER",
        3: "B-ORG",
        4: "I-ORG",
        5: "B-LOC",
        6: "I-LOC",
    }
    dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)
    data = []
    for text in dataset[split]:
        if len(data) > 10000:
            break
        for i in range(len(text["tokens"])):
            token = text["tokens"][i]
            tag = text["ner_tags"][i]
            if tag not in NER_tags:
                tag = NER_tags[0]
            else:
                tag = NER_tags[tag]
            data.append((token, tag))
    return data

You are asked to finish implementing the NERTagger class. You can modify the contents in this class, as long as you meet the requirements specified by the comments below, and your NERTagger can be used as

```
lm = TrigramLM("vocab.txt")
lm.train()
tagger = NERTagger(lm)
tagger.train()
ner_output = tagger.predict("John works at Microsoft and he loves it.")

```



In [46]:
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
import numpy as np


def get_training_data():
    return load_conll2003("train")


class NERTagger:
    def __init__(self, trained_lm):
        self.lm = trained_lm
        # For more documention on this usage, refer to
        # https://scikit-learn.org/dev/modules/generated/sklearn.pipeline.make_pipeline.html
        self.model = make_pipeline(DictVectorizer(), Perceptron(max_iter=1000, early_stopping=True), verbose=True)

    """
    TODO
    Trains the perceptron model on the training data.
    You will need to implement the missing part that extract the features labels based on the training data.
    """
    def train(self):
        # training_data returns a list of (token, label) tuples.
        training_data = get_training_data()
        features = []
        labels = []
        for i in range(len(training_data)):
            context = "" if i < 3 else f"{training_data[i-2][0]} {training_data[i-1][0]}"
            word, label = training_data[i]
            feature = self._extract_features(context, word) # Your code here
            features.append(feature)
            labels.append(label)

        self.model.fit(features, labels)


    """
    TODO
    Extracts features for the perceptron model.
    The features include:
    1) the top ten predictions (unordered) of the current word based on the context from your trained TrigramLM
    2) the current word
    Assume the context is "Jeff lives in" and the current_word is "Japan", and TrigramLM predict the top next words to be ["us", "england", ...]
    The features should include "us_in_next_word": True, "england_in_next_word": True, ..., "current_word_is_japan": True or "current_word": "Japan"
    @context : str : The previous context in the sentence
    @current_word : str : The current word to be predicted (the next word of the context, not included in @context)
    Returns: dict : A dictionary of features
    """
    def _extract_features(self, context, current_word, n=10):
        features = {}
        
        # Your code here
        top_words = self.lm.next_word_ranking(context, n)
        
        for word, _ in top_words:
            features[f"{word}_in_next_word"] = True

        features["current_word"] = current_word.lower()
        return features

    """
    TODO: fill some of the missing pieces.
    Predicts the named entities in the given sentence.
    @sentence : str : An input sentence for NER tagging
    Returns: list : A list of tuples containing the word and its predicted NER tag
    """
    def predict(self, sentence):
        self.train()
        words = self.lm.tokenize(sentence)
        tags = []

        for i in range(len(words)):
            context = "" if i < 3 else f"{words[i-2]} {words[i-1]}"
            current_word = words[i]
            feature = self._extract_features(context, current_word)
            tag = self.model.predict([feature])[0]
            tags.append((current_word, tag))

        return tags

## Task 3: Self Evaluation

Congratulations! You have completed all parts that are actually implementing the models. Now you need to do some testing to check if your implementation is correct.

Your implemented LM or tagger may not be very high-performing because of the limitations in model sizes and data sizes. There is no need to try to improve model performances as long as the basic implementation is correct. You will not receive any extra credit by improving the models.


In [47]:
# train language model and tagger
lm = TrigramLM("vocab.txt")
lm.train()
ner_tagger = NERTagger(lm)

vocab loaded, size = 26997
corpus loaded, size = 57340


In [48]:
'''
You MUST not modify any of the functions in this section, except self_evaluate().
DANGER: Modifying anything outside of self_evaluate will lead to an automatic zero for this assignment.
'''
def make_prediction_lm(lm, data):
    predictions = []
    for context in data:
        pred = lm.next_word_ranking(context)
        if len(pred) == 0:
            pred = []
        else:
            pred = [x[0] for x in pred]
        predictions.append(pred)
    return predictions

def make_prediction_tagger(tagger, text):
    pred = ner_tagger.predict(text)
    return [d[1] for d in pred]

# load eval news articles
def load_eval_news_data():
    texts = []
    with open("eval_news.txt", "r") as f:
        for line in f:
            texts.append(line.strip())
    data = []
    # tokenize and create a list of contexts
    for text in texts:
        tokens = word_tokenize(text.lower())
        for i in range(2, len(tokens)):
            context = " ".join(tokens[:i])
            data.append(context)
    print(f"data len = {len(data)}")
    return data

# run lm and tagger and generate prediction files.
def evaluate(lm, tagger):
    # evaluate lm
    pred_lm = make_prediction_lm(lm, load_eval_news_data())
    with open("lm_predictions.txt", "w") as f:
        for pred in pred_lm:
            f.write(" ".join(pred) + "\n")

    # evaluate tagger
    eval_ner_data = load_conll2003("test")[:200]
    text = " ".join([x[0] for x in eval_ner_data])
    pred_ner = make_prediction_tagger(tagger, text)
    with open("ner_predictions.txt", "w") as f:
        for pred in pred_ner:
            f.write(pred + "\n")

'''
Feel free to modify this part to check if your lm and NER tagger are doing the right thing.
Note that this part will not be graded.
'''
def self_evaluate(lm, tagger):
    # evaluate lm on toy data
    pred_lm = make_prediction_lm(lm, ["A", "A student", "A student at"])
    # expected output
    labl_lm = [['<UNK>', 'few', 'man', 'new', 'second', 'number', 'good', 'little', 'third', 'couple'],
            ['at', 'council', 'in', 'orator', 'was', 'of', 'you', 'organization', 'who', 'to'],
            ['the', '<UNK>', 'georgia', 'arms', 'harvard', 'brown']]
    assert len(pred_lm) == len(labl_lm)
    for i in range(3):
        print(pred_lm[i])
        print(labl_lm[i])
        #assert pred_lm[i] == labl_lm[i]

    # evaluate ner tagger
    pred_tagger = make_prediction_tagger(tagger, "Jeff lives in Japan")
    # expected output
    labl_tagger = ['B-PER', 'O', 'O', 'B-LOC']
    # assert pred_tagger == labl_tagger
    print("pred_tagger: " + str(pred_tagger) + "\nlabl_tagger: " + str(labl_tagger))
    print("self evaluation passed!")

In [49]:
# self evaluate
self_evaluate(lm, ner_tagger)

['<UNK>', 'few', 'man', 'second', 'new', 'number', 'good', 'third', 'little', 'year']
['<UNK>', 'few', 'man', 'new', 'second', 'number', 'good', 'little', 'third', 'couple']
['at', 'you', 'who', 'was', 'to', 'telling', 'organization', 'orator', 'of', 'manager']
['at', 'council', 'in', 'orator', 'was', 'of', 'you', 'organization', 'who', 'to']
['the', 'harvard', 'georgia', 'brown', 'arms', '<UNK>', '}', '{', 'zworykin', 'zurich']
['the', '<UNK>', 'georgia', 'arms', 'harvard', 'brown']
[Pipeline] .... (step 1 of 2) Processing dictvectorizer, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing perceptron, total=   0.1s
pred_tagger: [np.str_('O'), np.str_('O'), np.str_('O'), np.str_('B-LOC')]
labl_tagger: ['B-PER', 'O', 'O', 'B-LOC']
self evaluation passed!


The final step is to run the evaluate() function below to generate lm_predictions.txt and ner_predictions.txt. These two files will need to be submitted.

In [50]:
# generate prediction files
evaluate(lm, ner_tagger)

data len = 203
[Pipeline] .... (step 1 of 2) Processing dictvectorizer, total=   0.1s
[Pipeline] ........ (step 2 of 2) Processing perceptron, total=   0.1s


## Final Question
Is accuracy a good metric for NER? Why and why not? What other metric should we use to better evaluate model performances? Write your response below.

Accuracy is not the best metric for measuring NER quality. While evaluating accuracy the model faces problems of class imbalance (majority of tokens in datasets are labled as 'O') and poor performance on partial matching (for example, classifying B-PER but failing to recognize I-PER should result in the entire failure on the entity). 
It may be more productive to compute precision (if we are interested in the recognition on a particular class), recall (identifying all entites of a particular class) or f1-score.

## Final Submission
Please answer the final question above and submit the completed notebook with intermediate runnning logs, as well as lm_predictions.txt and ner_predictions.txt

